In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTNeoXForCausalLM, AutoConfig
from accelerate import init_empty_weights

checkpoint = "OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5"
cache_dir='cache'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, cache_dir=cache_dir)
model = GPTNeoXForCausalLM.from_pretrained(checkpoint, cache_dir=cache_dir).half().cuda()



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
config = AutoConfig.from_pretrained(checkpoint)

with init_empty_weights():
	model = AutoModelForCausalLM.from_config(config)
model.tie_weights()

In [3]:
from accelerate import load_checkpoint_and_dispatch

sharded_checkpoints_path = "cache/models--OpenAssistant--oasst-sft-4-pythia-12b-epoch-3.5/snapshots/c493ea33b1f86f09f0dd621d4e8c1d9d4a8453b9/pytorch_model.bin.index.json"
model = load_checkpoint_and_dispatch(
    model, sharded_checkpoints_path, device_map="sequential", no_split_module_classes=["GPTJBlock"]
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 from accelerate import load_checkpoint_and_dispatch                                          │
│   2                                                                                              │
│   3 sharded_checkpoints_path = "cache/models--OpenAssistant--oasst-sft-4-pythia-12b-epoch-3.     │
│ ❱ 4 model = load_checkpoint_and_dispatch(                                                        │
│   5 │   model, sharded_checkpoints_path, device_map="sequential",                                │
│   6 )                                                                                            │
│   7                                                                                              │
│                                                                                                  │
│ /home/sstest/.local/lib/python3.10/site-packages/accelerate/big_modeling.py:479 in               │
│ load_checkpoint_and_dispatch                                                                     │
│                                                                                                  │
│   476 │   │   )                                                                                  │
│   477 │   if offload_state_dict is None and "disk" in device_map.values():                       │
│   478 │   │   offload_state_dict = True                                                          │
│ ❱ 479 │   load_checkpoint_in_model(                                                              │
│   480 │   │   model,                                                                             │
│   481 │   │   checkpoint,                                                                        │
│   482 │   │   device_map=device_map,                                                             │
│                                                                                                  │
│ /home/sstest/.local/lib/python3.10/site-packages/accelerate/utils/modeling.py:935 in             │
│ load_checkpoint_in_model                                                                         │
│                                                                                                  │
│   932 │   │   │   │   │   module_name = ".".join(module_name.split(".")[:-1])                    │
│   933 │   │   │   │   if module_name == "" and "" not in device_map:                             │
│   934 │   │   │   │   │   # TODO: group all errors and raise at the end.                         │
│ ❱ 935 │   │   │   │   │   raise ValueError(f"{param_name} doesn't have any device set.")         │
│   936 │   │   │   │   param_device = device_map[module_name]                                     │
│   937 │   │   │   │                                                                              │
│   938 │   │   │   │   if param_device == "disk":                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: gpt_neox.layers.18.attention.bias doesn't have any device set.

In [3]:
inputs = tokenizer("<|prompter|>What is a meme, and what's the history behind this word?<|endoftext|><|assistant|>", return_tensors="pt")
inputs = inputs.to(0)
tokens = model.generate(**inputs, do_sample=True, temperature=0.9, max_length=100)
tokenizer.decode(tokens[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'What is a meme, and what\'s the history behind this word?A meme is a unit of cultural information that is passed from person to person within a culture. It can be a joke, video, image, or piece of content that is widely spread and often becomes associated with a particular idea or theme. The word "meme" was coined in the 1950s by Richard Dawkins to describe an idea that is passed from one individual to another within a culture. He'